# Update Locator

Open the ArcGIS Pro project file Arcgis_Pro_Locators on GISCLIENT2 server.

Copy the following code into the ArcGIS Pro Python window and hit enter twice to run:

In [ ]:
import os
import arcpy

# Name the folder where the new geodatabase is stored in the format DatabaseName_MMYY (ex. SQLDEV_0622)
FOLDER_NAME = "SQLDEV_1223"

# Path where the new folder is stored
FOLDER_PATH = r"C:/GIS/Locators/"+FOLDER_NAME

# Creates directory for folder
os.mkdir(FOLDER_PATH)

# Creates new file geodatabase in the new folder
arcpy.management.CreateFileGDB(FOLDER_PATH, "Locator_Data", "CURRENT")

# Loops through specific feature layers and converts them to point layers
point_layers = ["WATER_SERVICE", "WATER_MAIN", "SEWER_LINE"]
for layer in point_layers:
    point_output = FOLDER_PATH+"/Locator_Data.gdb/"+layer+"_POINT"
    arcpy.management.FeatureToPoint(layer, point_output, "CENTROID")

# Moves the remaining feature layers to the new geodatabase
fc_gdb_inputs = "CATCH_BASIN;FIRE_HYDRANT;MANHOLE;STREET_CENTERLINE;SYSTEM_VALVE"
fc_gdb_output = FOLDER_PATH+"/Locator_Data.gdb"
arcpy.conversion.FeatureClassToGeodatabase(fc_gdb_inputs, fc_gdb_output)

# Loops through all asset locator names, fills out the necessary parameters, and creates the locator
locator_ids = ['WATER_SERVICE_POINT', 'WATER_MAIN_POINT', 'SEWER_LINE_POINT', 'SYSTEM_VALVE', 'FIRE_HYDRANT', 'MANHOLE', 'CATCH_BASIN']
file_names = ['WaterService', 'WaterMain', 'SewerLine', 'SystemValve', 'FireHydrant', 'Manhole', 'CatchBasin']
for i in range(len(locator_ids)):
    primary_reference_data = locator_ids[i]+" POI"
    field_map = "'POI.FEATURE_ID "+locator_ids[i]+".FEATURE_ID';'POI.PLACE_NAME "+locator_ids[i]+".FACILITY_ID'"
    out_locator = FOLDER_PATH+"/"+file_names[i]
    arcpy.geocoding.CreateLocator("USA", primary_reference_data, field_map, out_locator, "ENG", None, None, None, "GLOBAL_HIGH")
    
# Sets up parameters for street address locator
street_primary_reference = "STREET_CENTERLINE StreetAddress"
street_field_map = "'StreetAddress.FEATURE_ID STREET_CENTERLINE.FEATURE_ID';\
                    'StreetAddress.HOUSE_NUMBER_FROM_LEFT STREET_CENTERLINE.MIN_ADDRESS';\
                    'StreetAddress.HOUSE_NUMBER_TO_LEFT STREET_CENTERLINE.MAX_ADDRESS';\
                    'StreetAddress.HOUSE_NUMBER_FROM_RIGHT STREET_CENTERLINE.MIN_ADDRESS';\
                    'StreetAddress.HOUSE_NUMBER_TO_RIGHT STREET_CENTERLINE.MAX_ADDRESS';\
                    'StreetAddress.PARITY_LEFT STREET_CENTERLINE.PARITY';\
                    'StreetAddress.PARITY_RIGHT STREET_CENTERLINE.PARITY';\
                    'StreetAddress.STREET_PREFIX_DIR STREET_CENTERLINE.STREET_PREFIX_CODE';\
                    'StreetAddress.STREET_NAME STREET_CENTERLINE.STREET_NAME';\
                    'StreetAddress.STREET_SUFFIX_TYPE STREET_CENTERLINE.STREET_TYPE_CODE';\
                    'StreetAddress.STREET_SUFFIX_DIR STREET_CENTERLINE.STREET_SUFFIX_CODE';\
                    'StreetAddress.FULL_STREET_NAME STREET_CENTERLINE.COMPLETE_STREET_NAME';\
                    'StreetAddress.CITY_LEFT STREET_CENTERLINE.NEIGHBORHOOD_DESC';\
                    'StreetAddress.CITY_RIGHT STREET_CENTERLINE.NEIGHBORHOOD_DESC'"
out_street_locator = FOLDER_PATH+"/StreetAddress"

# Creates StreetAddress locator
arcpy.geocoding.CreateLocator("USA", street_primary_reference, street_field_map, out_street_locator, "ENG", None, None, None, "GLOBAL_HIGH")

After the code above is done running, in the Catalog pane, right-click Folders > Add Folder Connection > Navigate to the newly created folder (DatabaseName_MMYY)

**Repeat manually for each new locator:**
1. In the Catalog pane, expand the new folder. Right-click a newly created locator and choose Locator Properties.
2. Click on Geocoding options in the left panel of the Locator Properties window.
3. For all EXCEPT StreetAddress (CatchBasin, Fire Hydrant, Manhole, SewerLine, SystemValve, WaterMain, WaterService):
    - set Minimum match score to 100 
4. For StreetAddress ONLY:
    - set Minimum candidate score to 35
    - set Side offset to 0
    - set Match with No Zones to Yes
5. Click OK.

Copy the code below into the Python window and run:

In [ ]:
# Selects all asset locators for the composite locator
address_locators = FOLDER_PATH+"/CatchBasin.loc CatchBasin;"+FOLDER_PATH+"/FireHydrant.loc FireHydrant;"+FOLDER_PATH+"/Manhole.loc Manhole;"+FOLDER_PATH+"/SewerLine.loc SewerLine;"+FOLDER_PATH+"/SystemValve.loc SystemValve;"+FOLDER_PATH+"/WaterService.loc WaterService;"+FOLDER_PATH+"/WaterMain.loc WaterMain"

# Field mapping split up into 10 parts
address1 = r'Address "Address or Place" true true false 100 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Address,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Address,0,0,'+FOLDER_PATH+'/Manhole.loc,Address,0,0,'+FOLDER_PATH+'/SewerLine.loc,Address,0,0,'+FOLDER_PATH+'/SystemValve.loc,Address,0,0,'+FOLDER_PATH+'/WaterService.loc,Address,0,0,'+FOLDER_PATH+'/WaterMain.loc,Address,0,0;'
address2 = r'Address2 "Address2" true true false 100 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Address2,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Address2,0,0,'+FOLDER_PATH+'/Manhole.loc,Address2,0,0,'+FOLDER_PATH+'/SewerLine.loc,Address2,0,0,'+FOLDER_PATH+'/SystemValve.loc,Address2,0,0,'+FOLDER_PATH+'/WaterService.loc,Address2,0,0,'+FOLDER_PATH+'/WaterMain.loc,Address2,0,0;'
address3 = r'Address3 "Address3" true true false 100 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Address3,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Address3,0,0,'+FOLDER_PATH+'/Manhole.loc,Address3,0,0,'+FOLDER_PATH+'/SewerLine.loc,Address3,0,0,'+FOLDER_PATH+'/SystemValve.loc,Address3,0,0,'+FOLDER_PATH+'/WaterService.loc,Address3,0,0,'+FOLDER_PATH+'/WaterMain.loc,Address3,0,0;'
neighborhood = r'Neighborhood "Neighborhood" true true false 50 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Neighborhood,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Neighborhood,0,0,'+FOLDER_PATH+'/Manhole.loc,Neighborhood,0,0,'+FOLDER_PATH+'/SewerLine.loc,Neighborhood,0,0,'+FOLDER_PATH+'/SystemValve.loc,Neighborhood,0,0,'+FOLDER_PATH+'/WaterService.loc,Neighborhood,0,0,'+FOLDER_PATH+'/WaterMain.loc,Neighborhood,0,0;'
city = r'City "City" true true false 50 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,City,0,0,'+FOLDER_PATH+'/FireHydrant.loc,City,0,0,'+FOLDER_PATH+'/Manhole.loc,City,0,0,'+FOLDER_PATH+'/SewerLine.loc,City,0,0,'+FOLDER_PATH+'/SystemValve.loc,City,0,0,'+FOLDER_PATH+'/WaterService.loc,City,0,0,'+FOLDER_PATH+'/WaterMain.loc,City,0,0;'
subregion = r'Subregion "County" true true false 50 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Subregion,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Subregion,0,0,'+FOLDER_PATH+'/Manhole.loc,Subregion,0,0,'+FOLDER_PATH+'/SewerLine.loc,Subregion,0,0,'+FOLDER_PATH+'/SystemValve.loc,Subregion,0,0,'+FOLDER_PATH+'/WaterService.loc,Subregion,0,0,'+FOLDER_PATH+'/WaterMain.loc,Subregion,0,0;'
region = r'Region "State" true true false 50 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Region,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Region,0,0,'+FOLDER_PATH+'/Manhole.loc,Region,0,0,'+FOLDER_PATH+'/SewerLine.loc,Region,0,0,'+FOLDER_PATH+'/SystemValve.loc,Region,0,0,'+FOLDER_PATH+'/WaterService.loc,Region,0,0,'+FOLDER_PATH+'/WaterMain.loc,Region,0,0;'
postal = r'Postal "ZIP" true true false 20 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,Postal,0,0,'+FOLDER_PATH+'/FireHydrant.loc,Postal,0,0,'+FOLDER_PATH+'/Manhole.loc,Postal,0,0,'+FOLDER_PATH+'/SewerLine.loc,Postal,0,0,'+FOLDER_PATH+'/SystemValve.loc,Postal,0,0,'+FOLDER_PATH+'/WaterService.loc,Postal,0,0,'+FOLDER_PATH+'/WaterMain.loc,Postal,0,0;'
postalext = r'PostalExt "ZIP4" true true false 20 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,PostalExt,0,0,'+FOLDER_PATH+'/FireHydrant.loc,PostalExt,0,0,'+FOLDER_PATH+'/Manhole.loc,PostalExt,0,0,'+FOLDER_PATH+'/SewerLine.loc,PostalExt,0,0,'+FOLDER_PATH+'/SystemValve.loc,PostalExt,0,0,'+FOLDER_PATH+'/WaterService.loc,PostalExt,0,0,'+FOLDER_PATH+'/WaterMain.loc,PostalExt,0,0;'
countrycode = r'CountryCode "Country" true true false 100 Text 0 0,First,#,'+FOLDER_PATH+'/CatchBasin.loc,CountryCode,0,0,'+FOLDER_PATH+'/FireHydrant.loc,CountryCode,0,0,'+FOLDER_PATH+'/Manhole.loc,CountryCode,0,0,'+FOLDER_PATH+'/SewerLine.loc,CountryCode,0,0,'+FOLDER_PATH+'/SystemValve.loc,CountryCode,0,0,'+FOLDER_PATH+'/WaterService.loc,CountryCode,0,0,'+FOLDER_PATH+'/WaterMain.loc,CountryCode,0,0'

# Combines all parts of the field map
composite_field_map = address1 + address2 + address3 + neighborhood + city + subregion + region + postal + postalext + countrycode

# Sets selection criteria and output
selection_criteria = "CatchBasin #;FireHydrant #;Manhole #;SewerLine #;SystemValve #;WaterService #;WaterMain #"
out_composite_locator = FOLDER_PATH+"/BWSC_Locator"

# Creates composite locator
arcpy.geocoding.CreateCompositeAddressLocator(address_locators, composite_field_map, selection_criteria, out_composite_locator)